Analyze zuora


In [14]:
var request = require('request');
var fs = require('fs');
var { URL } = require('url');

var zuoraOauth = (credentials) => {
    var rest_api, rest_user, rest_pass;
}

var httpRequest = (options) => {
    return new Promise((resolve, reject) => {
        request(options, function (error, res, body) {
            if(error) {
                reject(error);
            }
            
            res.setEncoding('utf8');
            const contentType = res.headers['content-type'];
            const location = res.headers['location'];
            
            if(typeof location !== 'undefined') {
                return httpRequest({
                    method: 'GET',
                    uri: location,
                })
                    .then(r => resolve(r))
            }
            resolve(body);
        });
        
        if(typeof body !== 'undefined') {
            req.write(body);
        }
    });
}

var token, expires;
var eloquaOauth = (eloquaConfig) => {
    var authBody = {
        "grant_type": "password",
        "scope": "full",
        "username": eloquaConfig.rest_api_company + '\\' + eloquaConfig.rest_api_user,
        "password": eloquaConfig.rest_api_password
    };
    return httpRequest({
        uri: eloquaConfig.token_uri,
        method: 'POST',
        json: authBody,
        headers: {
            'Authorization': "Basic " + new Buffer(eloquaConfig.rest_client_id + ":" + eloquaConfig.rest_secret).toString("base64")
        },
    })
        .then(newToken => {
            token = newToken.access_token;
            newToken['expires'] = expires = new Date((new Date()).getTime() + parseFloat(newToken.expires_in) * 1000);
            return newToken;
        })
}

var zuoraExport = (query) => {
    
}

var csvToJson = (csv) => {
    
}

var mapDataToFields = () => {
    
}

var eloquaBulkImport = (json) => {
    
}

module.exports = {
    eloquaBulkImport,
    mapDataToFields,
    csvToJson,
    zuoraExport,
    zuoraOauth,
    eloquaOauth
}


{ eloquaBulkImport: [Function: eloquaBulkImport],
  mapDataToFields: [Function: mapDataToFields],
  csvToJson: [Function: csvToJson],
  zuoraExport: [Function: zuoraExport],
  zuoraOauth: [Function: zuoraOauth],
  eloquaOauth: [Function: eloquaOauth] }

In [16]:

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var eloquaConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/eloqua_production.json').toString().trim());

var eloquaToken;
try {
    eloquaToken = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/eloqua_token.json').toString().trim());
} catch (e) {
}

$$.async();
Promise.resolve(typeof eloquaToken !== 'undefined'
                ? eloquaToken
                : module.exports.eloquaOauth(eloquaConfig))
    .then(r => {
        eloquaToken = r;
        fs.writeFileSync(
            PROFILE_PATH + '/.credentials/eloqua_token.json',
            JSON.stringify(r, null, 4));
        $$.sendResult(r);
    })
    .catch(e => $$.sendError(e))


{ access_token: 'MTk2Njk1MDY1NDoxNjlFS3BUajhsMEtWZ1FXckJFdkZaTzFjV1FXWU9tU1p3dldvU293aEV3TWJTU3J+SlZHTlV2MW5ldFVDZGNUZk5YMkwza29Uc2VueUw2VFhsQnp5TTgyUzVENml6dFp0NzFU',
  token_type: 'bearer',
  expires_in: 28800,
  refresh_token: 'MTk2Njk1MDY1NDoxd2RrRktvLTlVMFc4Nm8yZE5XNE5tY0RNWnZlLWlyOUp6MFU5cDRVa2pSNzRzRlZydUdidFo0VjZzWXBVb2ZscVVQTHEya21oMWxTbzQ0bWwwU2UxdnQycThzSlktdzRab1JW',
  expires: '2017-10-31T08:31:56.473Z' }

Test zuora


In [ ]:
import assert from 'assert';
var xlsx = require('xlsx');

/*import {
    eloquaBulkImport,
    mapDataToFields,
    csvToJson,
    zuoraExport,
    zuoraOauth,
    eloquaOauth
} from '';
*/
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var worksheet = PROFILE_PATH + '/Documents/asm/Marketing_File_Oct_.xslx';
var eloquaConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/eloqua_production.json').toString());
var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString());

describe('zuora to eloqua', () => {
    it('should transfer data end-to-end', () => {
        var success = eloquaBulkImport(
            mapDataToFields(
                csvToJson(
                    zuoraExport(
                        zuoraOauth()))), eloquaOauth());
        assert(success);
    })
    
    it('should have comparable records', () => {
        const workSheetsFromFile = XLSX.utils.sheet_to_json(worksheet);
        console.log(workSheetsFromFile);
    })
})

describe('zuora oauth', () => {
    it('should connect to zuora using oauth', () => {
        
    })
})

describe('zuora export', () => {
    it('should export a CSV from zuora', () => {
        
    })
    
    it('should convert CSV to JSON', () => {
        
    })
})

describe('map zuora data fields', () => {
    it('should map basic data', () => {
        
    })
    
    it('should map contact data', () => {
        
    })
    
    it('should map support data', () => {
        
    })
    
    it('should map cancelled data', () => {
        
    })
})

describe('eloqua bulk upload', () => {
    it('should authenticate using eloqua oauth', () => {
        console.log(eloquaOauth(eloquaConfig))
    })
    
    it('should update data to eloqua', () => {
        
    })
    
    it('should confirm upload was successful', () => {
        
    })
})


Zuora query


In [ ]:
var excludedProducts = [
    'Act! Pro - New License',
    'Act! Pro - 30 Day Support',
    'Act! Pro - Upgrade License',
    'Act! Password Reset Charge',
    'Act! Premium Cloud - Trial',
    'Act! Pro V19 - Upgrade License',
]
var excludedSkus = [
    '00000006'
]
var currencies = [
    'USD',
    'AUD',
    'NZD',
]


/*
GET_MODIFIED_SUBSCRIPTIONS,
from.ToString("yyyy-MM-dd"),
to.ToString("yyyy-MM-dd"),
DateTime.Now.ToString("yyyy-MM-dd"),
string.Join("' AND ProductRatePlan.Name!='", renewalsServiceConfiguration.ProductExclusions),
string.Join("' AND Product.SKU!='", renewalsServiceConfiguration.SkuExclusions),
string.Join("' OR Account.Currency='", renewalsServiceConfiguration.CurrencyInclusions)
*/

module.exports = `SELECT
    Account.Id,
    Account.Name,
    Account.AccountNumber,
    Account.resellerofRecord__c,
    Account.renewalRep__c,
    Account.commisionedSalesRep__c,
    Account.CreatedDate,
    SoldToContact.WorkEmail,
    SoldToContact.Country,
    BillToContact.WorkEmail,
    RatePlanCharge.Id,
    RatePlanCharge.BillingPeriod,
    RatePlanCharge.Description,
    RatePlanCharge.Quantity,
    RatePlanCharge.Version,
    RatePlanCharge.CreatedDate,
    RatePlanCharge.EffectiveEndDate,
    ProductRatePlan.planType__c,
    ProductRatePlan.planSubType__c,
    Product.productType__c,
    Subscription.Name,
    Subscription.Status,
    Subscription.Reseller__c,
    Subscription.SubscriptionEndDate,
    Subscription.SubscriptionStartDate,
    Subscription.TermStartDate,
    Subscription.TermEndDate,
    Subscription.AutoRenew
FROM RatePlanCharge
WHERE Subscription.Status!='Draft' AND Subscription.Status!='Expired'
    AND Subscription.TermEndDate &gt;='{0}' AND Subscription.TermEndDate &lt;='{1}'
    AND (Account.Currency='{5}')
    AND (Product.SKU!='{4}')
    AND (RatePlanCharge.EffectiveEndDate &gt;='{2}' OR RatePlanCharge.ChargeType='OneTime')
    AND (ProductRatePlan.Name!='{3}')
    AND RatePlanCharge.BillingPeriod!='Month'`
.replace('{3}', excludedRatePlans)
.replace('{4}', excludedProductSkus)
.replace('{5}', currencies)
